<div style="display:flex;">
<img alt="New Relic" style="display:block;height:64px" src="https://gitlab.com/softbutterfly/open-source/open-source-office/-/raw/master/banners/borderless/brands/new_relic.png" />
<img alt="SoftButterfly" style="display:block;height:64px;margin-left:auto" src="https://gitlab.com/softbutterfly/open-source/open-source-office/-/raw/master/banners/borderless/softbutterfly.png" />
</div>

# New Relic Playground

## Imports

Python Imports

In [ ]:
import os

Third-party libraries

In [ ]:
import dotenv
import requests
from newrelic_sb_sdk.client import NewRelicGqlClient
from newrelic_sb_sdk.utils.query import NULL_CURSOR
from newrelic_sb_sdk.utils.response import get_response_data, print_response

## Client setup

Load credentials

In [ ]:
env_file = "../.env"

dotenv.load_dotenv(env_file)

NEW_RELIC_USER_KEY = os.environ.get("NEW_RELIC_USER_KEY", None)
NEW_RELIC_ACCOUNT_ID = os.environ.get("NEW_RELIC_ACCOUNT_ID", None)

if NEW_RELIC_USER_KEY is None:
    raise ValueError("Environment variable NEW_RELIC_USER_KEY is not set.")

if NEW_RELIC_ACCOUNT_ID is None:
    raise ValueError("Environment variable NEW_RELIC_ACCOUNT_ID is not set.")

Client instantiation

In [ ]:
newrelic = NewRelicGqlClient(new_relic_user_key=NEW_RELIC_USER_KEY)

Define test query

In [ ]:
query = newrelic.build_query(
    """
        {
            actor {
                user {
                    email
                    name
                    id
                }
            }
        }
    """,
    {},
)

Executing query

In [ ]:
response = newrelic.execute(query)

Looking for response data

In [ ]:
print_response(response)

## Playground area

### List all available dashboards

In [ ]:
query_string = """
    {
        actor {
            entitySearch(
                queryBuilder: {
                    type: DASHBOARD
                }
            ) {
                count
                results(cursor: %(cursor)s) {
                    nextCursor
                    entities {
                        ... on DashboardEntityOutline {
                            guid
                            name
                            owner {
                                email
                                userId
                            }
                            indexedAt
                            permissions
                        }
                    }
                }
            }
        }
    }
"""

query_params = {
    "cursor": NULL_CURSOR,
}

query = newrelic.build_query(
    query_string=query_string,
    query_params=query_params,
)

In [ ]:
response = newrelic.execute(query)

In [ ]:
dashboard_count = get_response_data(
    response,
    key_path="entitySearch:count",
    action="actor",
)
dashboard_count

In [ ]:
dashboards_list = get_response_data(
    response,
    key_path="entitySearch:results:entities",
    action="actor",
)
len(dashboards_list)

In [ ]:
dashboards_list[0]

In [ ]:
next_cursor = get_response_data(
    response,
    key_path="entitySearch:results:nextCursor",
    action="actor",
)
next_cursor

### Get a single dashboard

In [ ]:
dashboard_guid = dashboards_list[0]["guid"]

In [ ]:
query_string = """
{
    actor {
        entity(guid: %(guid)s) {
            ... on DashboardEntity {
                guid
                name
                owner {
                    email
                    userId
                }
                permissions
                indexedAt
            }
        }
    }
}
"""

query_params = {
    "guid": f'"{dashboard_guid}"',
}

query = newrelic.build_query(
    query_string=query_string,
    query_params=query_params,
)

In [ ]:
response = newrelic.execute(query)

In [ ]:
dashboard = get_response_data(
    response,
    key_path="entity",
    action="actor",
)
dashboard

### Get a dashboard with pages

In [ ]:
dashboard_guid = dashboards_list[0]["guid"]

In [ ]:
query_string = """
{
    actor {
        entity(guid: %(guid)s) {
            ... on DashboardEntity {
                guid
                name
                owner {
                    email
                    userId
                }
                permissions
                indexedAt
                pages {
                    guid
                    name
                }
            }
        }
    }
}
"""

query_params = {
    "guid": f'"{dashboard_guid}"',
}

query = newrelic.build_query(
    query_string=query_string,
    query_params=query_params,
)

In [ ]:
response = newrelic.execute(query)

In [ ]:
dashboard = get_response_data(
    response,
    key_path="entity",
    action="actor",
)
dashboard

### Get a dashboard with pages and widget

In [ ]:
dashboard_guid = dashboards_list[0]["guid"]

In [ ]:
query_string = """
{
    actor {
        entity(guid: %(guid)s) {
            ... on DashboardEntity {
                guid
                name
                owner {
                    email
                    userId
                }
                permissions
                indexedAt
                pages {
                    guid
                    name
                    widgets {
                        id
                        rawConfiguration
                        title
                        visualization {
                            id
                        }
                    }
                }
            }
        }
    }
}
"""

query_params = {
    "guid": f'"{dashboard_guid}"',
}

query = newrelic.build_query(
    query_string=query_string,
    query_params=query_params,
)

In [ ]:
response = newrelic.execute(query)

In [ ]:
dashboard = get_response_data(
    response,
    key_path="entity",
    action="actor",
)
dashboard.keys()

In [ ]:
{key: value for key, value in dashboard.items() if key not in ("pages")}

In [ ]:
dashboard_pages = dashboard["pages"]
len(dashboard_pages)

In [ ]:
dashboard_page_0 = dashboard_pages[0]
dashboard_page_0.keys()

In [ ]:
{key: value for key, value in dashboard_page_0.items() if key not in ("widgets")}

In [ ]:
dashboard_page_0_widgets = dashboard_page_0["widgets"]
len(dashboard_page_0_widgets)

In [ ]:
dashboard_page_0_widget_0 = dashboard_page_0_widgets[0]
dashboard_page_0_widget_0

### Create a dashboard

In [ ]:
query_string = """
    mutation {
        dashboardCreate(
            accountId: %(account_id)d,
            dashboard: {
                name: "Sample api dashboard",
                pages: {
                    name: "Sample api page",
                    widgets: {
                        configuration: {
                            markdown: {
                                text: "# Sample api widget"
                            }
                        },
                        title: ""
                    }
                },
                permissions: PUBLIC_READ_WRITE
            }
        ) {
            entityResult {
                name
                owner {
                    email
                    userId
                }
                guid
            }
            errors {
                description
                type
            }
        }
    }
"""

query_params = {
    "account_id": NEW_RELIC_ACCOUNT_ID,
}

query = newrelic.build_query(
    query_string=query_string,
    query_params=query_params,
)

In [ ]:
response = newrelic.execute(query)

In [ ]:
errors = get_response_data(
    response,
    key_path="errors",
    action="dashboardCreate",
)
errors

In [ ]:
created_dashboard = get_response_data(
    response,
    key_path="entityResult",
    action="dashboardCreate",
)
created_dashboard

### Task: Create dashboard page snapshot

Read the documentation carefully and create a snapshot of an existing dashboard

In [ ]:
# Put your code here

# Define your query template
query_string = """
"""

# Define your query parameters if necessary
query_params = {}


query = newrelic.build_query(
    query_string=query_string,
    query_params=query_params,
)

response = newrelic.execute(query)

In [ ]:
# only interested on the dashboard url
snapshot_url = get_response_data(
    response,
    action="dashboardCreateSnapshotUrl",
)
print(snapshot_url)

# replace PDF with PNG, and get the link to download the file
url_png = snapshot_url[:-3] + "PNG"
print(url_png)

# rename the downloaded file, and save it in the working directory
dashboard_response = requests.get(
    url_png,
    stream=True,
)

with open("dashboard_example.png", "wb") as file_handler:
    file_handler.write(dashboard_response.content)